## 1. Data Exploration and Data Combination Process

This project used Fu, C. & Kazmi, H.'s method of combining 112 files into one CSV file stored in a pivot format for best performance (2022)

Fu, C. & Kazmi, H. (2022, November 10). [buildsys22 tutorial] 0 data preparation. Kaggle. URL: https://www.kaggle.com/code/patrick0302/buildsys22-tutorial-0-data-preparation

In [2]:
#import libraries
import boto3
import sagemaker
import pandas as pd
import os
import tqdm
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
s3_private_path_csv = "s3://{}/Smart_Grid/data/archive-2/halfhourly_dataset/halfhourly_dataset/".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-911199926915/Smart_Grid/data/archive-2/halfhourly_dataset/halfhourly_dataset/


Checking all the smart meter consumption load profile in the folder:

In [4]:
prefix = 'Smart_Grid/data/archive-2/halfhourly_dataset/halfhourly_dataset/'
file_names = []

s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

for page in page_iterator:
    if 'Contents' in page:
        for obj in page['Contents']:
            # Extract the file name from the key
            file_name = obj['Key'][len(prefix):]
            # Add the file name to the list
            file_names.append(file_name)

# Now file_names contains only the names of the files
print(file_names)

['block_0.csv', 'block_1.csv', 'block_10.csv', 'block_100.csv', 'block_101.csv', 'block_102.csv', 'block_103.csv', 'block_104.csv', 'block_105.csv', 'block_106.csv', 'block_107.csv', 'block_108.csv', 'block_109.csv', 'block_11.csv', 'block_110.csv', 'block_111.csv', 'block_12.csv', 'block_13.csv', 'block_14.csv', 'block_15.csv', 'block_16.csv', 'block_17.csv', 'block_18.csv', 'block_19.csv', 'block_2.csv', 'block_20.csv', 'block_21.csv', 'block_22.csv', 'block_23.csv', 'block_24.csv', 'block_25.csv', 'block_26.csv', 'block_27.csv', 'block_28.csv', 'block_29.csv', 'block_3.csv', 'block_30.csv', 'block_31.csv', 'block_32.csv', 'block_33.csv', 'block_34.csv', 'block_35.csv', 'block_36.csv', 'block_37.csv', 'block_38.csv', 'block_39.csv', 'block_4.csv', 'block_40.csv', 'block_41.csv', 'block_42.csv', 'block_43.csv', 'block_44.csv', 'block_45.csv', 'block_46.csv', 'block_47.csv', 'block_48.csv', 'block_49.csv', 'block_5.csv', 'block_50.csv', 'block_51.csv', 'block_52.csv', 'block_53.csv', '

In [5]:
#set list to hold halfhourlydata
halfhourlydataset = []
counter = 0
#loop to read every file to one list
for filename in tqdm.tqdm(file_names):
    dftemp = pd.read_csv(os.path.join(s3_private_path_csv, filename),index_col='tstp',parse_dates=True, low_memory=False)
    dftemp['filename'] = filename.split('.')[0]
    dftemp = dftemp.replace('Null', np.nan).dropna()
    dftemp['energy(kWh/hh)'] = dftemp['energy(kWh/hh)'].astype('float')
    halfhourlydataset.append(dftemp)
    
#load list to data frame  
halfhourlydataset = pd.concat(halfhourlydataset, axis=0)
halfhourlydataset

100%|██████████| 112/112 [12:38<00:00,  6.77s/it]


,LCLid,energy(kWh/hh),filename
tstp,,,
2012-10-12 00:30:00,MAC000002,0.000,block_0
2012-10-12 01:00:00,MAC000002,0.000,block_0
2012-10-12 01:30:00,MAC000002,0.000,block_0
2012-10-12 02:00:00,MAC000002,0.000,block_0
2012-10-12 02:30:00,MAC000002,0.000,block_0
...,...,...,...
2014-02-27 22:00:00,MAC005019,0.129,block_99
2014-02-27 22:30:00,MAC005019,0.095,block_99
2014-02-27 23:00:00,MAC005019,0.061,block_99


In [6]:
#store the data in a new CSV file for next processing
dfoutput = halfhourlydataset.groupby(['tstp', 'LCLid'])['energy(kWh/hh)'].mean().unstack()

Store the data to a CSV file

In [ ]:
from io import StringIO
csv_buffer = StringIO()
dfoutput.to_csv(csv_buffer)

object_key = 'Smart_Grid/data/processeddata/dataT.csv'

s3.put_object(Bucket=bucket, Key=object_key, Body=csv_buffer.getvalue())

In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>